Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Forecasting Pipeline
---

In this notebook we create a pipeline to forecast sales with the models we trained in the last step. The forecasting pipeline we'll set up is similar to the training pipeline in the last step so we'll keep the documentation light. For more details on the steps and functions refer to the last notebook.

### Prerequisites
At this point, you should have already:

1. Created your AML Workspace using the [00_Setup_AML_Workspace notebook](../00_Setup_AML_Workspace.ipynb)
2. Run [01_Data_Preparation.ipynb](01_Data_Preparation.ipynb) to setup your compute and create the dataset
3. Run [02_Training_Pipeline.ipynb](02_Training_Pipeline.ipynb) to train the models

## 1.0 Connect to workspace and datastore

In [ ]:
from azureml.core import Workspace
from azureml.core import Datastore

ws = Workspace.from_config()

# set up datastores
dstore = ws.get_default_datastore()

print('Workspace Name: ' + ws.name, 
      'Azure Region: ' + ws.location, 
      'Subscription Id: ' + ws.subscription_id, 
      'Resource Group: ' + ws.resource_group, sep='\n')

## 2.0 Create an experiment

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, 'forecasting_pipeline')

## 3.0 Get the dataset

In [ ]:
from azureml.core.dataset import Dataset

small_dataset = Dataset.get_by_name(ws, name='oj_data_small')
small_dataset_input = small_dataset.as_named_input('forecast_10_models')

## 4.0 Create ParallelRunStep for the forecasting pipeline
As we did with the training pipeline, we'll create a ParallelRunStep to parallelize our forecasting process. You'll notice this code is essentially the same as the last step except that we'll be parallelizing **forecast.py** rather than train.py.

### 4.1 Configure environment for ParallelRunStep

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

forecast_env = Environment(name="many_models_environment")
forecast_conda_deps = CondaDependencies.create(pip_packages=['sklearn', 'joblib'])
forecast_env.python.conda_dependencies = forecast_conda_deps

### 4.2 Choose a compute target

In [ ]:
from azureml.core.compute import AmlCompute
compute = AmlCompute(ws, "cpucluster")

### 4.3 Set up ParallelRunConfig

In [ ]:
from azureml.contrib.pipeline.steps import ParallelRunConfig 

process_count_per_node = 8
node_count = 5
timeout = 180

tags = {}
tags['node_count'] = node_count
tags['process_count_per_node'] = process_count_per_node
tags['timeout'] = timeout

parallel_run_config = ParallelRunConfig(
    source_directory='./scripts',
    entry_script='forecast.py',
    mini_batch_size='1',
    run_invocation_timeout=timeout, 
    error_threshold=10,
    output_action='append_row', 
    environment=forecast_env, 
    process_count_per_node=process_count_per_node, 
    compute_target=compute, 
    node_count=node_count
)

### 4.4 Set up ParallelRunStep

In [ ]:
from azureml.pipeline.core import PipelineData
from azureml.contrib.pipeline.steps import ParallelRunStep 

output_dir = PipelineData(name='forecasting_output', datastore=dstore)

parallel_run_step = ParallelRunStep(
    name="many-models-forecasting",
    parallel_run_config=parallel_run_config,
    inputs=[small_dataset_input],
    output=output_dir,
    allow_reuse=False,
    arguments=['--forecast_horizon', 8,
              '--starting_date', '1992-10-01',
              '--target_column', 'Quantity',
              '--timestamp_column', 'WeekStarting',
              '--model_type', 'lr',
              '--date_freq', 'W-THU']
)

## 5.0 Create step to copy predictions

The forecasting pipeline includes a second step that copies the predictions from *parallel_run_step.txt* to a CSV file in a separate container. While this step is simple, it demonstates how a step can be added to the pipeline to upload the predictions to a separate datastore or make additional transformations to the output.

### 5.1 Create a data reference
First, we create a datastore named **predictions** to hold the outputs of the pipeline and get a reference to it:

In [ ]:
from azureml.data.data_reference import DataReference

output_dstore = Datastore.register_azure_blob_container(
    workspace=ws, 
    datastore_name="predictions",
    container_name="predictions",
    account_name=dstore.account_name,
    account_key=dstore.account_key,
    create_if_not_exists=True
)

output_dref = DataReference(output_dstore)

### 5.2 Create PythonScriptStep
Next, we define the [PythonScriptStep](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.python_script_step.pythonscriptstep?view=azure-ml-py) and give it our newly create datastore as well as the location of the *parallel_run_step.txt*:

In [ ]:
from azureml.pipeline.steps import PythonScriptStep

upload_predictions_step = PythonScriptStep(
    name="copy_predictions",
    script_name="copy_predictions.py",
    compute_target=compute,
    source_directory='./scripts',
    inputs=[output_dref, output_dir],
    allow_reuse=False,
    arguments=['--parallel_run_step_output', output_dir,
              '--output_dir', output_dref]
)

## 6.0 Run the pipeline

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallel_run_step, upload_predictions_step])
run = experiment.submit(pipeline, tags=tags)

## 7.0 View the results of the forecasting pipeline
To see our forecasts, we download the *parallel_run_step.txt*, read the results into a dataframe, and visualize the predictions. Note that we could also download the results from the predictions container we created above.

### 7.1 Download parallel_run_step.txt locally

In [ ]:
import os

def download_predictions(run, target_dir=None):
    stitch_run = run.find_step_run("many-models-forecasting")[0]
    port_data = stitch_run.get_output_data('forecasting_output')
    print(port_data)
    port_data.download(target_dir, show_progress=True)
    step_hash = os.listdir(os.path.join(target_dir, 'azureml'))[0]
    return  os.path.join(target_dir, 'azureml', step_hash, 'forecasting_output')

file_path = download_predictions(run, 'output')
file_path

### 7.2 Convert the file to a dataframe

In [ ]:
import pandas as pd

df = pd.read_csv(file_path + '/parallel_run_step.txt', sep=" ", header=None)
df.columns = ['WeekStarting', 'Predictions', 'Store', 'Brand']
df['WeekStarting'] = [d.date() for d in pd.to_datetime(df['WeekStarting'])]
df.head()

### 7.3 Visualize the predictions
First, we look at the distribution of predicted quantities by brand:

In [ ]:
fig = sns.violinplot(x=df['Brand'], y=df['Predictions'], data=df)
fig.set_title('Predictions by Brand')

Then we look at those predictions over time:

In [ ]:
import matplotlib.pyplot as plt

week = df.groupby(['WeekStarting', 'Brand'])
week = week['Predictions'].sum()
week = pd.DataFrame(week.unstack(level=1))

week.plot()
plt.title('Total Predictions by Brand')
plt.xticks(rotation=40)
plt.legend(loc='upper right')
plt.xlabel('Week')
plt.ylabel('Total Predictions')
plt.show()

From there, we can trim the results to look at individual brands:

In [ ]:
store = 'Store1001'
df_1001 = df[df['Store'] == store]

brands = df_1001.groupby(['WeekStarting','Brand'])
brands= brands['Predictions'].sum()
brands= pd.DataFrame(brands.unstack(level=1))

brands.plot()
plt.legend(loc='upper right', labels=brands.columns.values)
plt.xticks(rotation=40)
plt.title('Predictions for Store 1001')
plt.xlabel('Week')
plt.ylabel('Predicted Quantity')
plt.show()

## 8.0 Publish and schedule the pipeline (Optional)


### 8.1 Publish the pipeline
Once you have a pipeline you're happy with, you can publish a pipeline so you can call it programatically later on. See this [tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-your-first-pipeline#publish-a-pipeline) for additional information on publishing and calling pipelines.

In [ ]:
# published_pipeline = pipeline.publish(name = 'forecast_many_models',
#                                      description = 'forecast many models',
#                                      version = '1',
#                                      continue_on_step_failure = False)

### 8.2 Schedule the pipeline
You can also [schedule the pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-schedule-pipelines) to run on a time-based or change-based schedule. This could be used to automatically retrain models every month or based on another trigger such as data drift.

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
# training_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Week", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="forecasting_pipeline_recurring_schedule", 
#                             description="Schedule Forecasting Pipeline to run on the first day of every week",
#                             pipeline_id=training_pipeline_id, 
#                             experiment_name=experiment.name, 
#                             recurrence=recurrence)